In [1]:
from sage.misc.prandom import randint
from sage.rings.all import ZZ, IntegerModRing, RR, PolynomialRing, GF
from sage.arith.all import next_prime, euler_phi, crt
from sage.misc.functional import cyclotomic_polynomial, round
from random import randint

# Parameter Generation

In [2]:
d = 3
m = 3

In [3]:
log_q = 5
primes = [2]
q = 2
for i in range(1,log_q):
    p = next_prime(primes[-1])
    primes.append(p)
    q *= p

In [ ]:
#testin cell
q =29

In [4]:
N = 2**5
var('Z')
g = cyclotomic_polynomial(N, var='Z')

Z_q = IntegerModRing(q)
Q = Z_q['Z'].quo(g,Z)

In [5]:
N

32

In [6]:
D = d*m
var('Y')
monomials = [Z_q['Z','Y'](Y+1)]
h = Q['Y'](Y+1)
for i in range(2,D):
    mon = Q['Y'](Y+i)
    monomials.append(mon)
    h *= mon

R = Z_q['Z','Y'].quo(g,Z).quo(h,Y)

# BV11

In [ ]:
s = Q.random_element()

In [ ]:
def encrypt(s,mu):
    assert mu in Z_q
    assert s in Q
    a = R(Q.random_element())
    
    # TODO make the noise real
    e = randint(0,2)

    b = a*R(s)+d*e+mu
    return -a*R(Y)+b

In [ ]:
c1 = encrypt(s,1)
c2 = encrypt(s,2)
c = c1 * c2

In [ ]:
def decrypt(s,c):
    a = Q(c.lift()(Y=s)).lift()(Z=0).lift()
    return mod(a,d)

In [ ]:
decrypt(s,c)

# Database Interpolation

In [7]:
P = PolynomialRing(GF(d),d,"x")
x = P.gens()

In [8]:
#TODO multivariate polynomial interpolation

In [9]:
f = P(x[0]*x[1]*2+x[1]*3+x[0]*4+x[2]*x[1]*5)
f

-x0*x1 - x1*x2 + x0

# Eval

In [ ]:
def normal_eval(f,cs):
    f_prime =f.change_ring(R)
    return f_prime(cs)

In [ ]:
normal_eval(f,[c1,c2,c1])

In [ ]:
assert q.is_prime()
def split_eval_R(f,cs):
    print(Z_q.is_field())
    """
    ys = []
    for mon in monomials:
        R_reduced = Z_q['Z'],'Y'].quo(g,Z).quo(mon,Y)
        f_prime = f.change_ring(R_reduced)
        cs_reduced = [R_reduced(c.lift()) for c in cs]
        y_prime = f_prime(cs_reduced).lift()
        ys.append(y_prime)
    print(monomials[0].parent())
    print(monomials[0])
    return crt(ys,monomials)
    """
split_eval_R(f,[c1,c2,c1])

# Testing Eval

In [ ]:
def normal_eval_Z_q(f,cs):
    f_prime =f.change_ring(Z_q)
    return f_prime(cs)

In [ ]:
h

In [ ]:
h_prime = h.change_ring(Z_q)
h_prime

In [ ]:
normal_eval_Z_q(f,[Z_q(700),Z_q(200),Z_q(23)])

In [ ]:
def split_eval_Z_q(f,cs):
    ys = []
    for prime in primes:
        Z_p = IntegerModRing(prime)
        f_prime = f.change_ring(Z_p)
        cs_reduced = [Z_p(c.lift()) for c in cs]
        y_prime = f_prime(cs_reduced).lift()
        ys.append(y_prime)
    return Z_q(crt(ys,primes))
split_eval_Z_q(f,[Z_q(700),Z_q(200),Z_q(23)])

## next

In [10]:
def to_Z_qY(mon):
    return Z_q['Y'](Z_q['Z','Y'](mon))

In [11]:
h_prime = to_Z_qY(h)

In [12]:
R_test = Z_q['Y'].quo(h_prime)

In [13]:
def normal_eval_Z_qY(f,cs):
    f_prime =f.change_ring(Z_q['Y'].quo(h_prime))
    return f_prime(cs)

In [17]:
normal_eval_Z_qY(f,[Z_q['Y'](Y+700),Z_q['Y'](Y + 200),Z_q['Y'](23)])

2*Ybar^2 + 1847*Ybar + 1150

In [15]:
monomials

[Y + 1, Y + 2, Y + 3, Y + 4, Y + 5, Y + 6, Y + 7, Y + 8]

In [ ]:
def split_eval_Z_qY(f,cs):
    ys = []
    monomials_prime = [to_Z_qY(mon) for mon in monomials]
    for mon in monomials_prime:
        Z_qY_prime = Z_q['Y'].quo(to_Z_qY(mon))
        f_prime = f.change_ring(Z_qY_prime)
        #print(cs[0].parent())
        cs_reduced = [Z_qY_prime(c.lift()) for c in cs]
        y_prime = f_prime(cs_reduced).lift()
        ys.append(y_prime)
    return crt(ys,monomials_prime)
split_eval_Z_qY(f,[Z_q['Y'].quo(h_prime)(Y+700),Z_q['Y'].quo(h_prime)(Y + 200),Z_q['Y'].quo(h_prime)(23)])

In [4]:
var('Y')

Y

In [7]:
H = IntegerModRing(20)['Y']

In [ ]:
crt([H(7),H(8),H(6)],[H(Y+1),H(Y+2),H(Y+3)])